## DNN predicts accuracy

In [2]:
from pathlib import Path
import tensorflow as tf
import tensorflow.keras as K
import numpy as np
import pandas as pd
from tqdm import tqdm

import gtda.diagrams as D

In [3]:
dgms_path = Path() / "cifar10_wt_udgms_l01.npz"; assert dgms_path.exists()
metrics_path = Path.home() / 'data' / 'cnn_zoo' / 'cifar10' / 'metrics.csv'; assert metrics_path.exists()
kSEED = 2021

In [4]:
with np.load(dgms_path) as f:
    counts = f['counts']
    udgms = f['udgms']
    
df = pd.read_csv(metrics_path)
metrics = df[df.step==86].copy()
df = 0

In [5]:
if 'ix' not in metrics:
    metrics['ix'] = list(range(len(metrics)))
    metrics = metrics[metrics.train_accuracy > 0.2]
    
    v=np.array(metrics.ix)
    counts = counts[v]
    udgms = udgms[v]
    
metrics

,Unnamed: 0,config.activation,config.b_init,config.dataset,config.dnn_architecture,config.dropout,config.epochs,config.epochs_between_checkpoints,config.init_std,config.l2reg,...,config.train_fraction,config.w_init,modeldir,step,test_accuracy,test_loss,train_accuracy,train_loss,laststep,ix
8,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,relu,zeros,cifar10,cnn,0.456778,86,20,0.009018,4.206822e-07,...,1.00,glorot_normal,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,86,0.4762,1.491586,0.474690,1.484884,True,0
17,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,tanh,zeros,cifar10,cnn,0.450656,86,20,0.002582,1.512473e-05,...,0.10,RandomNormal,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,86,0.2749,1.996954,0.277127,1.984243,True,1
26,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,tanh,zeros,cifar10,cnn,0.589027,86,20,0.035074,5.094971e-08,...,0.25,TruncatedNormal,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,86,0.2941,1.899296,0.305176,1.889742,True,2
62,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,relu,zeros,cifar10,cnn,0.182887,86,20,0.231639,2.636589e-03,...,0.10,RandomNormal,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,86,0.3313,1.976165,0.333116,1.973141,True,6
71,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,tanh,zeros,cifar10,cnn,0.524521,86,20,0.001087,9.241976e-03,...,0.50,TruncatedNormal,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,86,0.2233,2.195609,0.221761,2.198477,True,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269954,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,tanh,zeros,cifar10,cnn,0.518933,86,20,0.003196,1.966038e-07,...,0.50,TruncatedNormal,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,86,0.3420,1.797064,0.337769,1.794660,True,29994
269963,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,tanh,zeros,cifar10,cnn,0.609046,86,20,0.073117,4.484191e-08,...,0.50,he_normal,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,86,0.2229,2.144266,0.215373,2.151409,True,29995
269972,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,relu,zeros,cifar10,cnn,0.087478,86,20,0.205680,1.207098e-08,...,0.10,RandomNormal,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,86,0.2422,2.095041,0.259549,2.089865,True,29996
269981,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,relu,zeros,cifar10,cnn,0.490268,86,20,0.005778,3.226268e-03,...,0.10,orthogonal,/cns/ym-d/home/brain-ber/dnn_science/unterthin...,86,0.3254,1.988230,0.322483,1.988447,True,29997


In [6]:
# dmgs = [np.repeat(udgm, cnt, axis=0) for cnt, udgm in zip(counts, udgms)]
# len(dgms), dgms[20].shape

udgm = udgms[0]
count = counts[0]

dgm = np.repeat(udgm, count, axis=0)

udgm.shape, count.shape, dgm.shape, sum(count)

((3349, 3), (3349,), (126166, 3), 126166)

In [7]:
np.random.seed(kSEED)
assert len(metrics.step) == len(counts)
assert len(metrics.step) == len(udgms)
N = len(udgms)
train_size = round(N/2)
perm = np.random.permutation(N)
spliter = lambda v: (v[perm[:train_size]], v[perm[train_size:]])

train_y, valid_y = spliter(np.asarray(metrics.test_accuracy))
# train_X, valid_X = spliter(udgms)

In [8]:
udgm.shape

(3349, 3)

In [9]:
from joblib import Parallel, delayed
pi = D.Silhouette(n_bins=256*4, n_jobs=1, power=0.3)

def dgm2pi(udgm, cnt):
    r = np.repeat(udgm, np.clip(
        cnt,
        0,
        4# 3000,
    ), axis=0)
    # return r
    return pi.fit_transform([r])[0]

l = list(zip(udgms, counts))
# udgms = counts = 0

b = 8
result = Parallel(n_jobs=-1)(
    delayed(dgm2pi)(udgm_, cnt) for udgm_, cnt in tqdm(l))


100%|██████████| 21882/21882 [01:22<00:00, 264.89it/s]


In [10]:
train_X, valid_X = spliter(np.asarray(result))

In [11]:
# model = K.models.Sequential([
#     K.layers.Dense(2048),
#     K.layers.BatchNormalization(),
#     K.layers.ReLU(),
    
#     K.layers.Dense(2048),
#     K.layers.BatchNormalization(),
#     K.layers.ReLU(),
    
#     K.layers.Dense(2048),
#     K.layers.BatchNormalization(),
#     K.layers.ReLU(),
    
#     K.layers.Dense(1, activation=tf.nn.sigmoid),
# ])


np.random.seed(2021)
tf.random.set_seed(2021)


W = 512*4
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(W, activation='relu'),
    tf.keras.layers.Dense(W, activation='relu'),
    # tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(W, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(W, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(W, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])


batch_size = 128
lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=len(train_X)/batch_size,
    decay_rate=0.96,
    staircase=True,
)


model.compile(
    optimizer=tf.keras.optimizers.Adam(lr_scheduler),
    loss='mse',
    metrics=['mse', 'mae']
)


hist = model.fit(
    train_X, train_y,
    epochs=25,
    validation_data=(valid_X, valid_y),
)

2022-01-06 10:30:24.946016: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-06 10:30:25.016674: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-06 10:30:25.016821: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-06 10:30:25.017749: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/25
 41/342 [==>...........................] - ETA: 1s - loss: 0.0078 - mse: 0.0078 - mae: 0.0694

2022-01-06 10:30:27.396234: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


342/342 [==============================] - 3s 5ms/step - loss: 0.0049 - mse: 0.0049 - mae: 0.0555 - val_loss: 0.0041 - val_mse: 0.0041 - val_mae: 0.0516
Epoch 2/25
342/342 [==============================] - 2s 5ms/step - loss: 0.0041 - mse: 0.0041 - mae: 0.0508 - val_loss: 0.0042 - val_mse: 0.0042 - val_mae: 0.0526
Epoch 3/25
342/342 [==============================] - 2s 5ms/step - loss: 0.0038 - mse: 0.0038 - mae: 0.0488 - val_loss: 0.0039 - val_mse: 0.0039 - val_mae: 0.0505
Epoch 4/25
342/342 [==============================] - 2s 5ms/step - loss: 0.0037 - mse: 0.0037 - mae: 0.0478 - val_loss: 0.0035 - val_mse: 0.0035 - val_mae: 0.0463
Epoch 5/25
342/342 [==============================] - 2s 4ms/step - loss: 0.0036 - mse: 0.0036 - mae: 0.0469 - val_loss: 0.0035 - val_mse: 0.0035 - val_mae: 0.0464
Epoch 6/25
342/342 [==============================] - 2s 5ms/step - loss: 0.0035 - mse: 0.0035 - mae: 0.0462 - val_loss: 0.0035 - val_mse: 0.0035 - val_mae: 0.0469
Epoch 7/25
342/342 [=======

In [12]:
best_score = min(hist.history['val_mse'])
best_score

0.0031874352134764194

In [13]:
r2 = 1 -  best_score/np.var(valid_y)
print(f'{r2:.03f}')

0.478


In [14]:
print(f'{(1-0.987)*np.var(valid_y):.07f}')

0.0000793
